In [9]:
import os, yaml
import requests
from openai import OpenAI
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.embeddings import OpenAIEmbeddings



In [7]:
#Setting the Credentials of OpenAI API
with open('credentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']

In [20]:
def complete_model(USER_MESSAGE):
    response = client.chat.completions.create(
                                            model = 'gpt-4o',
                                            messages = [
                                                        {"role": "system", "content" : "You are a helpful assitant to identify the tag for sense of a word"},
                                                        {"role": "user", "content": USER_MESSAGE}              
                                                        ],
                                            temperature=0,
                                            max_tokens=1500
                                            )
    return str(response.choices[0].message.content)

In [24]:

embedding_llm =  OpenAIEmbeddings(model="text-embedding-3-small")


In [12]:
#Creating the tex Chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
                                        separator="\n",
                                        chunk_size=1000,
                                        chunk_overlap=200,
                                        )
    chunks = text_splitter.split_text(text)
    return chunks

In [25]:
#buidling the vector store
def get_vectorstore(text_chunks):
    vectorstore = FAISS.from_texts(
                                    texts=text_chunks, 
                                    embedding=embedding_llm
                                    )
    return vectorstore

In [26]:
def get_data():
    file= open("data.txt","r")
    txt=file.read()
    
    return txt

In [27]:
def data_pipeline():
    text = get_data()
    chunks = get_text_chunks(text)
    vectorstore = get_vectorstore(chunks)
    return vectorstore

In [28]:
vectorstore=data_pipeline()

In [29]:
#Not Required for the porgram. Displaying the relevant documents from Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k" : 3}, search_type="similarity")


In [ ]:

docs = retriever.get_relevant_documents("")
docs

In [ ]:
#Finetuned UMberto model
from transformers import pipeline
classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-emotion',top_k=2)
prediction = classifier(docs)
print(prediction)


In [ ]:
#GPT 4o
prompt=" Given the test, assign a emotional label from [] "